In [1]:
import numpy as np

# Part 1

From chatGPT:

With slight modifications in the `__repr__ ` method

In [2]:
from typing import List, Type

class Node:
    def __init__(self, file_type: str, name: Type, size: int, parent: Type["Node"] = None):
        assert file_type == 'dir' or file_type == 'file'
        self.type = file_type
        self.name = name
        self.size = size
        self.parent = parent
        self.children = []

    def add_child(self, child: Type["Node"]):
        self.children.append(child)
        child.parent = self

    def get_parent(self) -> Type["Node"]:
        return self.parent

    def __repr__(self, level: int = 0) -> str:
        if self.type == 'dir':
            ret = "    "*level + "└-- " + self.name + "\n"
        elif self.type == 'file':
            ret = "    "*level + "└-- " + self.name + " " + repr(self.size) + "\n"
        for child in self.children:
            ret += child.__repr__(level + 1)
        return ret
        

Testing the `Node` class

In [3]:
file_system = Node('dir', '/', 0)

file_system.add_child(Node('file', 'ptsd', 5325))
file_system.add_child(Node('file', 'bfdf', 355))
dir1 = Node('dir', 'dir1', 0)
file_system.add_child(dir1)
dir1.add_child(Node('file', 'ghods', 56372))
dir1.add_child(Node('file', 'bfd', 372))
dir2 = Node('dir', 'dir2', 0)
file_system.add_child(dir2)
dir2.add_child(Node('file', 'bfd', 372))

In [4]:
file_system

└-- /
    └-- ptsd 5325
    └-- bfdf 355
    └-- dir1
        └-- ghods 56372
        └-- bfd 372
    └-- dir2
        └-- bfd 372

Implement the logic of constructing the FileSystem from the input

In [5]:
def read_file_system(input_file):
    # Open the input file for reading
    with open(input_file, "r") as f:
        lines = f.read().splitlines()

    # Create the root directory
    root = Node('dir', '/', 0)
    curr_dir = root

    i = 0
    while i <= len(lines):
        line = lines[i]
        if line.startswith('$'):
            # This means it's a command to either change directory, or to list the contents
            tokens = line.split(" ")[1:]
            if tokens[0] == 'cd':
                # Change directory
                if tokens[1] == '/':
                    # Go to root directory
                    curr_dir = root
                elif tokens[1] == '..':
                    # Go to parent directory
                    curr_dir = curr_dir.get_parent()
                else:
                    # Enter specified directory from the childs
                    for child in curr_dir.children:
                        if child.name == tokens[1]:
                            curr_dir = child
                            break
                i += 1

            elif tokens[0] == 'ls':
                # Show contents of current Node and use this to create new files or directories
                j = 1
                tmp_line = lines[i+j]
                while not tmp_line.startswith('$'):
                    tmp_tokens = tmp_line.split(' ')
                    if tmp_tokens[0] == 'dir':
                        curr_dir.add_child(Node('dir', tmp_tokens[1], 0))
                    else:
                        curr_dir.add_child(Node('file', tmp_tokens[1], int(tmp_tokens[0])))
                    j += 1
                    # Catch possibility of reaching end of input file
                    if i+j < len(lines):
                        tmp_line = lines[i+j]
                    else:
                        i += j
                        break
                i += j
    
    return root

In [6]:
test_root = read_file_system('test_input.txt')
test_root

└-- /
    └-- a
        └-- e
            └-- i 584
        └-- f 29116
        └-- g 2557
        └-- h.lst 62596
    └-- b.txt 14848514
    └-- c.dat 8504156
    └-- d
        └-- j 4060174
        └-- d.log 8033020
        └-- d.ext 5626152
        └-- k 7214296

### With real input file

In [7]:
real_root = read_file_system('input.txt')
real_root

└-- /
    └-- ccjp
        └-- dlz 159990
        └-- mbtsvblj
            └-- frqsf.nsv 34806
            └-- ppq
                └-- dhzp 266252
            └-- ptht
                └-- jbnj
                    └-- clscr
                        └-- dhzp 249531
                    └-- zwtwf.zfz 145780
                └-- zcbnwhzd
                    └-- mbtsvblj
                        └-- pjhvzjt.brz 258527
            └-- rgmvdwt
                └-- hdb 166021
                └-- ljpzpdbf 115350
        └-- nppbjl.qhg 165076
    └-- hglnvs.bsh 328708
    └-- hpsnpc
        └-- bfdfbrh.vff 6334
        └-- qphlw.whm 311949
    └-- pcb
        └-- ccjp
            └-- ccjp 10932
            └-- jjcggmr.ljs 80040
            └-- jrqwhsn.ptv 299982
            └-- ljpzpdbf 280655
            └-- mdvvj.dlf 268430
            └-- mvhhlg
                └-- fwhcbfr.shd 146902
            └-- ppfq
                └-- hdb 263452
            └-- rdgngdm
                └-- wlgshdgl 46355
    

Actually solve the problems question

In [40]:
dirs_at_most_100k = []
sizes_part1 = []

def size_of_dir(root):
    # Figure out total size of a directory recursively
    total_size = 0
    for child in root.children:
        if child.type == 'dir':
            total_size += size_of_dir(child)
        else:
            total_size += child.size
    return total_size

def find_dirs_at_most_100k(root):
    # Find all directories that are at most 100k
    if size_of_dir(root) <= 100000:
        dirs_at_most_100k.append(root)
        # Also save the sizes of each dir in sizes
        sizes_part1.append(size_of_dir(root))
    for child in root.children:
        if child.type == 'dir':
            find_dirs_at_most_100k(child)

In [41]:
find_dirs_at_most_100k(real_root)

In [42]:
sizes_part1

[46355,
 62811,
 43177,
 79532,
 90381,
 59872,
 90781,
 79812,
 34771,
 34771,
 27497,
 27497,
 27497,
 82162,
 79744,
 90698,
 73809,
 39489,
 43495,
 10176,
 22089,
 75969,
 14247,
 75000,
 75000,
 75000,
 52067]

In [44]:
sum(sizes_part1)

1513699

# Part 2

In [37]:
size_of_dir(real_root)

46876531

In [29]:
40_000_000 - size_of_dir(real_root)

-6876531

In [34]:
# Find the directory with a size at least 3123469
# By looking through the entire real_root
dirs_at_least = []
minimum_size = 6_876_531
def find_dirs_at_least(root, size):
    if size_of_dir(root) >= size:
        dirs_at_least.append(root)
    for child in root.children:
        if child.type == 'dir':
            find_dirs_at_least(child, size)

find_dirs_at_least(real_root, minimum_size)

In [35]:
# Calculate size of all these directories
sizes_part2 = []
for dir in dirs_at_least:
    sizes_part2.append(size_of_dir(dir))

In [46]:
min(sizes_part2)

7991939